<a href="https://colab.research.google.com/github/stevenbowler/EmployeeSurvey/blob/main/EmployeeSurvey6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploads direct from Google drive

In [ ]:
# 1. Install
!apt-get update -qq
!apt-get install -y poppler-utils -qq
!pip install -q opencv-python pandas matplotlib openai

In [ ]:
# 2. Imports & secrets
import os, glob, cv2, numpy as np, json, re
from pathlib import Path
from collections import defaultdict
from openai import OpenAI
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import userdata, files

In [ ]:
# Upload a folder (or zip) of PNGs
uploaded = files.upload()  # or drag-drop a folder

# Or mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# PNG_ROOT_DIR = "/content/drive/MyDrive/survey_pngs"

PNG_ROOT_DIR = userdata.get('PNG_TARGET_PATH')  # change if you mounted Drive

# Recursively find all PNGs in subfolders
png_paths = sorted(glob.glob(f"{PNG_ROOT_DIR}/**/*.png", recursive=True))
print(f"Found {len(png_paths)} PNG files in subfolders")

In [1]:
# Group PNGs by survey (page001 + page002)
from collections import defaultdict

survey_dict = defaultdict(dict)  # survey_name → {1: path, 2: path}

pattern = re.compile(r"(.+)_page(\d{3})\.png$", re.IGNORECASE)

for p in png_paths:
    m = pattern.match(Path(p).name)
    if m:
        name, page_num = m.group(1), int(m.group(2))
        survey_dict[name][page_num] = p

# Keep only surveys with both pages
valid_surveys = {k: v for k, v in survey_dict.items() if 1 in v and 2 in v}
print(f"Found {len(valid_surveys)} complete surveys (both pages)")

NameError: name 're' is not defined

In [ ]:
# 5. Exact ROI coordinates (measured on your PNGs)
# DPI = 300 → image size ≈ 2480 × 3500 px
COLS = {
    'A': (750, 38, 38),   # x, w, h
    'B': (910, 38, 38),
    'C': (1070, 38, 38),
    'D': (1230, 38, 38)
}

# Y positions for Q1-20 (page 1)
Y_PAGE1 = [
    208, 253, 298, 343, 388,
    433, 478, 523, 568, 613,
    658, 703, 748, 793, 838,
    883, 928, 973,1018,1063
]

# Build ROIs for page 1
ROIS_PAGE1 = {}
for q, y in enumerate(Y_PAGE1, 1):
    ROIS_PAGE1[q] = {
        'A': (COLS['A'][0], y, COLS['A'][1], COLS['A'][2]),
        'B': (COLS['B'][0], y, COLS['B'][1], COLS['B'][2]),
        'C': (COLS['C'][0], y, COLS['C'][1], COLS['C'][2]),
        'D': (COLS['D'][0], y, COLS['D'][1], COLS['D'][2]),
    }

# Page 2 (Q21-23) – same spacing
ROIS_PAGE2 = {
    21: {k: (v[0], 208, v[1], v[2]) for k, v in ROIS_PAGE1[1].items()},
    22: {k: (v[0], 253, v[1], v[2]) for k, v in ROIS_PAGE1[1].items()},
    23: {k: (v[0], 298, v[1], v[2]) for k, v in ROIS_PAGE1[1].items()},
}

In [ ]:
# 6. Robust preprocessing
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eq = cv2.equalizeHist(gray)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(eq)
    kernel = np.ones((5,5), np.uint8)
    closed = cv2.morphologyEx(enhanced, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.adaptiveThreshold(closed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 41, 12)
    return thresh

In [ ]:
# 7. Detect filled box
def is_filled(roi, thresh, ratio=0.35):
    x, y, w, h = roi
    crop = thresh[y:y+h, x:x+w]
    if crop.size == 0: return False
    return np.sum(crop == 0) / (w*h) >= ratio

In [ ]:
# 8. Extract answers from PNG pair
def extract_from_pngs(page1_path, page2_path):
    answers = ['X'] * 23

    # Page 1
    img1 = cv2.imread(page1_path)
    thresh1 = preprocess(img1)
    for q in range(1, 21):
        rois = ROIS_PAGE1[q]
        for choice in 'ABCD':
            if is_filled(rois[choice], thresh1):
                answers[q-1] = choice
                break

    # Page 2
    img2 = cv2.imread(page2_path)
    thresh2 = preprocess(img2)
    for q in range(21, 24):
        rois = ROIS_PAGE2[q]
        for choice in 'ABCD':
            if is_filled(rois[choice], thresh2):
                answers[q-1] = choice
                break

    return answers

In [ ]:
# 9. Run on all surveys
all_answers = []
for name, pages in valid_surveys.items():
    print(f"Processing {name}")
    ans = extract_from_pngs(pages[1], pages[2])
    all_answers.append(ans)
    print(" →", "".join(ans))

print(f"\nParsed {len(all_answers)} surveys")

Expected output for Accounting_1
 → ABABCBDXAADBCAXXACBDBAD

In [ ]:
# 10 Aggregate percentages
stats = defaultdict(lambda: defaultdict(int))
for row in all_answers:
    for q, ch in enumerate(row, 1):
        if ch != 'X':
            stats[q][ch] += 1

percentages = {}
for q, cnt in stats.items():
    tot = sum(cnt.values())
    per = {c: round(v/tot*100, 1) for c, v in cnt.items()}
    percentages[q] = dict(sorted(per.items(), key=lambda x: -x[1]))

print("\nRESULTS:")
for q in sorted(percentages):
    print(f"Q{q:02d}: {percentages[q]}")

In [ ]:
#  11. One-call Grok-4 summary
XAI_API_KEY = userdata.get('XAI_API_KEY')
client = OpenAI(api_key=XAI_API_KEY, base_url="https://api.x.ai/v1")

summary_prompt = f"""You are a data-analyst.
Aggregated results from {len(all_answers)} employees (23 questions, A=Strongly Agree … D=Strongly Disagree).

Percentages (top first):
{json.dumps(percentages, indent=2)}

Write a concise markdown report with:
1. Overall insights
2. Table of top answer per question
3. Patterns / anomalies
4. Visual description
5. 3 actionable recommendations
"""

resp = client.chat.completions.create(
    model="grok-4",
    messages=[{"role":"user","content":summary_prompt}],
    max_tokens=1500,
    temperature=0.3
)
print("\n### Grok-4 Report")
print(resp.choices[0].message.content)

In [ ]:
# 12. Charts + CSV
fig, axs = plt.subplots(5, 5, figsize=(20, 20))
axs = axs.ravel()
colors = {'A':'#2ca02c','B':'#1f77b4','C':'#ff7f0e','D':'#d62728'}
for i, (q, per) in enumerate(sorted(percentages.items())):
    ch, val = list(per.keys()), list(per.values())
    axs[i].bar(ch, val, color=[colors.get(c,'gray') for c in ch])
    axs[i].set_title(f'Q{q}')
    axs[i].set_ylim(0,100)
for ax in axs[len(percentages):]: ax.axis('off')
plt.tight_layout()
plt.show()

df = pd.DataFrame.from_dict(percentages, orient='index').fillna(0)
csv_path = "survey_results_png.csv"
df.to_csv(csv_path)
files.download(csv_path)
print(f"CSV saved → {csv_path}")

In [ ]:
# Debug
def debug_png(png_path):
    img = cv2.imread(png_path)
    thresh = preprocess(img)
    rois = ROIS_PAGE1 if 'page001' in png_path else ROIS_PAGE2
    start_q = 1 if 'page001' in png_path else 21
    end_q = 20 if 'page001' in png_path else 23
    for q in range(start_q, end_q + 1):
        r = rois[q]
        for ch in 'ABCD':
            x,y,w,h = r[ch]
            filled = is_filled((x,y,w,h), thresh)
            color = (0,255,0) if filled else (0,0,255)
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)
            cv2.putText(img, f"{q}{ch}", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    from google.colab.patches import cv2_imshow
    cv2_imshow(img)

debug_png(valid_surveys[list(valid_surveys.keys())[0]][1])  # page 1